In [44]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from sklearn.ensemble import IsolationForest
from tensorflow.keras.models import load_model

In [45]:
# Define your folder structure
data_dir = 'dataset'
classes = ['akash', 'shubham']

# Load and preprocess audio data
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []
    
    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                # Perform preprocessing (e.g., convert to Mel spectrogram and resize)
                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
                data.append(mel_spectrogram)
                labels.append(i)
    
    return np.array(data), np.array(labels)

# Split data into training and testing sets
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))  # Convert labels to one-hot encoding
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create a neural network model
input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(input_layer, output_layer)

In [46]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 2.5097 - accuracy: 0.5312 - val_loss: 390.8824 - val_accuracy: 0.3750
Epoch 2/200
1/1 [==============================] - 0s 53ms/step - loss: 339.2290 - accuracy: 0.4688 - val_loss: 110.5350 - val_accuracy: 0.3750
Epoch 3/200
1/1 [==============================] - 0s 59ms/step - loss: 92.1446 - accuracy: 0.4688 - val_loss: 0.4412 - val_accuracy: 0.3750
Epoch 4/200
1/1 [==============================] - 0s 55ms/step - loss: 0.3979 - accuracy: 0.4688 - val_loss: 0.6931 - val_accuracy: 0.6250
Epoch 5/200
1/1 [==============================] - 0s 56ms/step - loss: 0.6931 - accuracy: 0.5312 - val_loss: 0.6930 - val_accuracy: 0.6250
Epoch 6/200
1/1 [==============================] - 0s 55ms/step - loss: 0.6931 - accuracy: 0.5312 - val_loss: 0.6929 - val_accuracy: 0.6250
Epoch 7/200
1/1 [==============================] - 0s 96ms/step - loss: 0.6931 - accuracy: 0.5312 - val_loss: 0.6928 - val_accuracy: 0.6250
Epoch 8/200
1/1

In [48]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.875


In [49]:
# Save the model
model.save('audio_classification_model.h5')

In [64]:
# Load the saved model
model = load_model('audio_classification_model.h5')

# Define the target shape for input spectrograms
target_shape = (128, 128)

# Define your class labels
classes = ['akash', 'shubham']

# Function to preprocess and classify an audio file
def test_audio(file_path, model):
    # Load and preprocess the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
    mel_spectrogram = tf.reshape(mel_spectrogram, (1,) + target_shape + (1,))
    
    # Make predictions
    predictions = model.predict(mel_spectrogram)
    
    # Get the class probabilities
    class_probabilities = predictions[0]
    
    # Get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)
    
    return class_probabilities, predicted_class_index

# Test an audio file
# test_audio_file = 'testing/akash/output_segment_4.wav'
test_audio_file = 'testing/vejit.wav'
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

# Display results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

# Calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')

1/1 [==============================] - 0s 73ms/step
Class: akash, Probability: 0.4270
Class: shubham, Probability: 0.5730
The audio is classified as: shubham
Accuracy: 0.5730
